In [1]:
#!pip -q install bunkatopics --upgrade
!pip -q install git+https://github.com/charlesdedampierre/BunkaTopics.git@dev --upgrade

In [2]:
from bunkatopics import Bunka
from langchain_community.embeddings import HuggingFaceEmbeddings
import os

#model_name = "sentence-transformers/distiluse-base-multilingual-cased-v2"
#model_name = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
model_name = "OrdalieTech/Solon-embeddings-large-0.1"

embedding_model = HuggingFaceEmbeddings(model_name=model_name,# We recommend starting with a small model
                                        model_kwargs={"device": "cuda"}, # Or cuda if you have GPU
                                        encode_kwargs={"show_progress_bar": True}, # Show the progress of embeddings
                                        multi_process=False)  # set to True if you have mutliprocessing

In [3]:
import pandas as pd
#docs = pd.read_csv("https://raw.githubusercontent.com/regicid/gender_le_monde/main/test_bunka.csv")
df_docs = pd.read_csv("lemonde_bunka.csv")

df_docs_sample = df_docs.sample(100000, random_state=42)
full_docs = list(df_docs_sample['titles'])
print(len(full_docs))

100000


In [15]:
df_docs_sample = df_docs_sample.drop_duplicates('titles', keep='first')

,Unnamed: 0,0,titles,start_text
2950278,2950278,1997_10_17_discretement-le-gouvernement-a-prev...,"Discrètement, le gouvernement a prévu de relev...",//IMAGINE-T-ON qu'un gouvernement puisse décid...
68863,68863,2000_12_27_guide_3709684_1819218.txt,GUIDE,//TROUVER SON FILM Tous les films Paris et rég...
966132,966132,2016_04_08_le-censi-bouvard_4898885_4891617.txt,Le Censi-Bouvard,Ce régime concerne les bailleurs sous le s...
893547,893547,2014_03_27_pour-en-finir-avec-un-air-meurtrier...,Pour en finir avec un air meurtrier,"De 2008 à 2011, les chercheurs du réseau Aphek..."
2690246,2690246,1991_08_10_italie-scenes-dramatiques-dans-les-...,ITALIE scènes dramatiques dans les ports d'Otr...,Les autorités italiennes ont commencé à ra...
...,...,...,...,...
2319373,2319373,1981_02_09_l-innovation_2708816_1819218.txt,L'innovation,"//Comme son cirque périclitait, il avait imagi..."
2601154,2601154,1989_04_25_sept-elections-cantonales-partielle...,Sept élections cantonales partielles Recul des...,//Pour trois des sept élections cantonales org...
1689129,1689129,1963_11_30_les-cortes-creent-un-tribunal-d-ord...,Les Cortes créent un tribunal d'ordre public,"//Madrid, 29 novembre."
1712155,1712155,1964_08_03_l-impossible-devaluation-de-la-lire...,"L'"" impossible "" dévaluation de la lire",Le nouveau gouvernement italien aura-t-il ...


In [4]:
bunka = Bunka(embedding_model=embedding_model, language='french') # You can choose any language you prefer
bunka.fit(full_docs)

2024-01-29 13:43:09 - Bunka - INFO - Embedding documents... (can take varying amounts of time depending on their size)


Batches:   0%|          | 0/1303 [00:00<?, ?it/s]

Batches:   0%|          | 0/1303 [00:00<?, ?it/s]

Batches:   0%|          | 0/520 [00:00<?, ?it/s]

2024-01-29 13:50:30 - Bunka - INFO - Reducing the dimensions of embeddings...
2024-01-29 13:51:40 - Bunka - INFO - Extracting meaningful terms from documents...
100%|██████████| 99964/99964 [13:11<00:00, 126.37it/s] 


In [9]:
import jsonlines  # You may need to install this library using pip

# Dump the data into JSONL files
with jsonlines.open("bunka_docs_lemonde.jsonl", mode="w") as writer:
    for item in bunka.docs:
        writer.write(item.dict())


# Dump the data into JSONL files
with jsonlines.open("bunka_terms_lemonde.jsonl", mode="w") as writer:
    for item in bunka.terms:
        writer.write(item.dict())
"""
with jsonlines.open("bunka_topics_lemonde.jsonl", mode="w") as writer:
    for item in bunka.topics:
        writer.write(item.dump())"""

'\nwith jsonlines.open("bunka_topics_lemonde.jsonl", mode="w") as writer:\n    for item in bunka.topics:\n        writer.write(item.dump())'

In [14]:
bunka.terms[0]

Term(term_id='M', lemma='m', ent='', ngrams=1, count_terms=7906)

In [ ]:
"""df_topics = pd.DataFrame([x.dict() for x in bunka.topics])
df_topics.to_csv('topics.csv')

bunka.df_top_docs_per_topic_.to_csv('top_docs_per_topics.csv')

df_docs = pd.DataFrame([x.dict() for x in bunka.docs])
df_docs.to_csv('docs.csv')"""

In [6]:
#import hdbscan
#custom_clustering_model = hdbscan.HDBSCAN(min_cluster_size = 3000)

In [22]:
for topic in bunka.topics:
    try:
        topic.name = ' | '.join(topic.name.split(' | ')[:4])
    except:
        topic.name = topic.name
bunka.topics[0]

Topic(topic_id='bt-0', name='PRISON | JUSTICE | POLICE | procès', lemma_name=None, x_centroid=1.3274933878670425, y_centroid=0.7608452482656533, size=16187, top_doc_id=None, top_term_id=None)

In [45]:
"""import os
import pickle

# Define the path to your repository directory
repo_dir = '.'

# Create the repository directory if it doesn't exist
if not os.path.exists(repo_dir):
    os.makedirs(repo_dir)

# Save the 'bunka' instance as a pickle file
pickle_file = os.path.join(repo_dir, 'bunka.pickle')

with open(pickle_file, 'wb') as f:
    pickle.dump(bunka, f)"""

"import os\nimport pickle\n\n# Define the path to your repository directory\nrepo_dir = '.'\n\n# Create the repository directory if it doesn't exist\nif not os.path.exists(repo_dir):\n    os.makedirs(repo_dir)\n\n# Save the 'bunka' instance as a pickle file\npickle_file = os.path.join(repo_dir, 'bunka.pickle')\n\nwith open(pickle_file, 'wb') as f:\n    pickle.dump(bunka, f)"